In [1]:
import pandas as pd
import numpy as np
import pickle

pd.set_option('display.max_columns', None)

In [2]:
# import data

usa_tweets = pd.read_pickle("../usa_tweets_cleaned.pkl")
demo = pd.read_csv("../og datasets/county_statistics.csv")
us_data = pd.read_csv('../notebooks/uscities.csv')

Add counties column to USA Tweets

In [3]:
col_list = ['city', 'state_name', 'county_name']
us_data = us_data.loc[:, col_list]

In [4]:
us_data.rename(columns={'state_name':'state'}, inplace=True)

In [5]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [6]:
#Map a dictionary of abbreviations to the dataframe
us_data['state'] = us_data['state'].map(us_state_to_abbrev).fillna(us_data['state'])

In [7]:
us_data['city'] = us_data['city'].astype(str)
usa_tweets['city'] = usa_tweets['city'].astype(str)

us_data['state'] = us_data['state'].astype(str)
usa_tweets['state'] = usa_tweets['state'].astype(str)

In [8]:
df = pd.merge(usa_tweets, us_data, how='left', on=['city','state'])

In [10]:
df['county_name'].isna().sum()
df.county_name.fillna(df.city, inplace=True)
df = df.drop_duplicates(subset='id').reset_index(drop=True)

Clean up demographics dataset

In [12]:
# drop unneccessary columns

cols = [0,27,28,30]
demo.drop(demo.columns[cols], axis=1, inplace=True)


demo = demo.drop(demo.loc[:,'total_votes16':'votes16_Hillary_Clinton'].columns, axis = 1)
demo = demo.drop(demo.loc[:,'total_votes20':'deaths'].columns, axis = 1)
demo = demo.drop(demo.loc[:,'ChildPoverty':'FamilyWork'].columns, axis = 1)

In [13]:
# create new variables

demo['democrats'] = demo[['percentage16_Hillary_Clinton', 'percentage20_Joe_Biden']].mean(axis=1)
demo['republicans'] = demo[['percentage16_Donald_Trump', 'percentage20_Donald_Trump']].mean(axis=1)
demo['democrats'] = round(demo['democrats']*100,1)
demo['republicans'] = round(demo['republicans']*100,1)
demo["women"] = round(demo["Women"] * 100 / demo["TotalPop"],1)
demo["other_ethnicities"] = demo["Native"] + demo["Asian"] + demo["Pacific"]
demo['voting_age_citizens'] = round(demo['VotingAgeCitizen']*100/demo['TotalPop'],1)

In [14]:
# drop old columns

cols = [2,3,4,5,7,12,13,14,15]
demo.drop(demo.columns[cols], axis=1, inplace=True)

In [15]:
# make location column with county and state in it

demo["location"] = demo["county"] + ', ' + demo["state"]
df["location"] = df["county_name"] + ', ' + df["state"]

In [16]:
data = pd.merge(df, demo, on="location",how='left')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172068 entries, 0 to 172067
Data columns (total 28 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   hashtags             85840 non-null   object 
 1   favorite_count       172068 non-null  int64  
 2   id                   172068 non-null  int64  
 3   lang                 172068 non-null  object 
 4   place                122241 non-null  object 
 5   retweet_count        172068 non-null  int64  
 6   text                 172068 non-null  object 
 7   user_location        158275 non-null  object 
 8   city                 172068 non-null  object 
 9   state_x              172068 non-null  object 
 10  tweet_proc_length    172068 non-null  int64  
 11  county_name          172068 non-null  object 
 12  location             172068 non-null  object 
 13  county               157885 non-null  object 
 14  state_y              157885 non-null  object 
 15  TotalPop         

In [19]:
# drop unneeded columns

cols = [0,1,3,4,5,7,8,9,10,11,13,14,16]
data.drop(data.columns[cols], axis=1, inplace=True)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172068 entries, 0 to 172067
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   172068 non-null  int64  
 1   text                 172068 non-null  object 
 2   location             172068 non-null  object 
 3   TotalPop             153009 non-null  float64
 4   Hispanic             153009 non-null  float64
 5   White                153009 non-null  float64
 6   Black                153009 non-null  float64
 7   IncomePerCap         153009 non-null  float64
 8   Poverty              153009 non-null  float64
 9   Unemployment         153009 non-null  float64
 10  democrats            155283 non-null  float64
 11  republicans          155283 non-null  float64
 12  women                153009 non-null  float64
 13  other_ethnicities    153009 non-null  float64
 14  voting_age_citizens  153009 non-null  float64
dtypes: float64(12), i

In [22]:
data = data.dropna().reset_index(drop=True)

In [24]:
data

,id,text,location,TotalPop,Hispanic,White,Black,IncomePerCap,Poverty,Unemployment,democrats,republicans,women,other_ethnicities,voting_age_citizens
0,965587085993365505,ll juice left carrot tonight fresh juice morni...,"Miami-Dade, FL",2702602.0,67.5,13.7,16.3,25481.0,19.0,7.4,58.5,40.1,51.5,1.6,58.4
1,930124070238040066,climate fact course warm year concern learn cl...,"Spokane, WA",490764.0,5.4,85.3,1.6,28325.0,15.2,6.3,44.0,50.3,50.4,3.9,75.2
2,932917776783757312,well start plant gogreen eco optoutside actonc...,"Miami-Dade, FL",2702602.0,67.5,13.7,16.3,25481.0,19.0,7.4,58.5,40.1,51.5,1.6,58.4
3,942159912121102336,planet great winner world move ahead actonclim...,"District of Columbia, DC",672391.0,10.7,36.0,46.9,50832.0,17.4,8.0,93.4,4.0,52.6,3.9,74.8
4,949634237153595393,climatechange challenge urban challenge watch ...,"District of Columbia, DC",672391.0,10.7,36.0,46.9,50832.0,17.4,8.0,93.4,4.0,52.6,3.9,74.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152940,1210060038380740608,savage energy partner record break fiscal quar...,"Harris, TX",4525519.0,42.2,30.6,18.5,30856.0,16.8,6.4,55.0,42.3,50.3,7.1,57.6
152941,996746444211011586,hard tell snakeoil chemtrail globalwarmingisah...,"Lane, OR",363471.0,8.4,82.6,1.0,27032.0,18.8,7.6,58.0,36.4,50.7,3.5,77.7
152942,948537051418214400,standard winter hurricane warn part florida yi...,"Pitkin, CO",17747.0,9.8,85.7,1.2,65800.0,7.1,4.3,72.5,23.8,47.2,1.6,76.3
152943,947614408108335104,happy new yeaя fan globalwarmingisahoax hoax f...,"New York, NY",1653877.0,26.1,46.8,12.5,69529.0,17.3,6.2,85.9,12.2,52.7,12.0,71.6


In [25]:
# Save file

data.to_pickle("usa_tweets_demo.pkl")